In [ ]:
#EXAMPLES OF ALL MODELS

#lin_model = lay.linear_layer((10,5,True,True))
#ffmodel = lay.FeedForward([(10,8,True,True),(8,5,True,True)])
#att_model = lay.attention((24,5,4))
#skipatt, skipff = lay.get_coder_blocks(24,6,4,20)
#enc_model = lay.Encoder(skipatt, skipff)
#dec_model = lay.Decoder(skipatt, skipff)
#encstack_model = lay.EncoderStack(skipatt, skipff,4)
#transformer_layer = lay.TransformerLayer(skipatt, skipff, 4)
#model_list = [lin_model, ffmodel, att_model, skipatt, skipff, enc_model, dec_model, encstack_model, transformer_layer]

In [1]:
import torch
from torch import nn
import sys
from itertools import chain
import copy
import os
import math
from library_model import layers as lay
from library_model import model_building as mb
from data import data_loading as dt
from data import text as txt
import numpy as np
import random


Synthetic text data

In [2]:
mytextpiece = "<start> When I was young, I never needed anyone and making love was just for fun. Those days are gone! Living alone, I think of all the friends I've known but when I dial the telephone nobody is home. All by myself, I don't wanna be all by myself, anymore! <end>"
mytranslationpiece = " <start> Οταν ημουν νεος, δεν χρειαζομουν κανενα και το να κανω ερωτα ηταν απλα για ευχαριστηση. Αυτες οι μερες εχουν περασει! Ζωντας μονος μου, σκεφτομαι ολους τους φιλους που ξερω αλλα οταν τους καλω στο τηλεφωνο κανεις δεν ειναι σπιτι. Ολομοναχος, δε θελω πια να ειμαι ολομοναχος!"
mytext = [mytextpiece for _ in range(2000)]
mytranslation = [mytranslationpiece for _ in range(2000)]

In [3]:
tokenizer_in, tokenizer_out, dataloader, starting_wrd, ending_wrd = txt.get_tranformer_data(mytext, mytext, 80)
encode=tokenizer_in.text_encoding
decode=tokenizer_out.text_decoding
len(tokenizer_in.token_encoding.keys())

47

In [4]:
data = iter(dataloader)
(x,y) = next(data)
print(decode(x[0][0]))
print(decode(x[1][0]))
print(decode(y[0]))

<start> When I was young, I never needed anyone and making love was just for fun. Those days are gone! Living alone, I think of all the friends I've known but when I dial the telephone nobody is home. All by myself, I don't wanna be all by myself, anymore! <end>
<start> When I was young, I never needed anyone and making love was just for fun. Those days are gone! Living alone, I think of all the friends I've known but when I dial the telephone nobody is home. All by myself, I don't wanna be all by myself, anymore!
When I was young, I never needed anyone and making love was just for fun. Those days are gone! Living alone, I think of all the friends I've known but when I dial the telephone nobody is home. All by myself, I don't wanna be all by myself, anymore! <end>


Define transformer model

In [5]:
model, opt = mb.get_registered_Transformer_model(in_vocab_size=47, out_vocab_size=47, dim_in = 60, dim_key= 10, heads = 6, dim_internal=80, copies = 3, lr = 1., start=starting_wrd, end = ending_wrd)
model_operate = mb.NN_operating_tools(model, opt, learning_rate_schedule=mb.learning_rate_function(60, 2.5, 8))

In [6]:
x[0][0].unsqueeze(0).size()

torch.Size([1, 63])

In [7]:
model.eval()
decode(model.autoregression(x[0][0].unsqueeze(0), 62, mode="beam"))

'<start> be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be'

In [8]:
out, prob =model([x[0],x[1]])
decode(out[10])

'be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be'

In [9]:
model_operate.fit_transformer(15, dataloader, tokenizer=tokenizer_out, sample_input=x[0][0].unsqueeze(0))

Epoch: 0 -- time: 18.4728000164032 -- loss = 139.25997924804688

<start> be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be
Epoch: 1 -- time: 17.102269887924194 -- loss = 139.0017547607422

<start> be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be
Epoch: 2 -- time: 16.845630168914795 -- loss = 138.75772094726562

<start> be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be
Epoch: 3 -- time: 18.683647871017456 -- loss = 138.53318786621094

<start> be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be be b

In [ ]:
model_operate.save_pt("saved_models/random.pt")

In [45]:
out, prob =model([x[0][0].unsqueeze(0),x[1][0,:60].unsqueeze(0)])
decode(out[0])

"When I was young, I never needed anyone and making love was just for fun. Those days are gone! Living alone, I think of all the friends I've known but when I dial the friends nobody is home. All by myself, I don't wanna be all by myself, anymore"